In [ ]:
import numpy as np
import seaborn as sns
import pandas as pd
import itertools 
import csv
import collections
import matplotlib.pyplot as plt
import re
import nltk
from bs4 import BeautifulSoup
from gensim.models import word2vec

from sklearn.manifold import TSNE


sns.set_context("paper")
%matplotlib inline

RES_DIR = "../input/"
# Load train data (skips the content column)
def load_train_data():
    categories = ['cooking', 'robotics', 'travel', 'crypto', 'diy', 'biology']
    train_data = []
    for cat in categories:
        data = pd.read_csv("{}{}.csv".format(RES_DIR, cat), usecols=['id', 'title', 'tags','content'])
        data['category'] = cat
        train_data.append(data)
    
    return pd.concat(train_data)
data = load_train_data()
print(data.head())

test = pd.read_csv("../input/test.csv")
test['tags'] = ''
test['category'] = 'physics'
data=data.append(test)
del test
#data=data.sample(1200)

In [ ]:
uri_re = r'(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:\'".,<>?«»“”‘’]))'

def stripTagsAndUris(x):
    if x:
        # BeautifulSoup on content
        soup = BeautifulSoup(x, "html.parser")
        # Stripping all <code> tags with their content if any
        if soup.code:
            soup.code.decompose()
        # Get all the text out of the html
        text =  soup.get_text()
        # Returning text stripping out all uris
        return re.sub(uri_re, "", text)
    else:
        return ""

# This could take a while
data["title"] = data["title"].map(stripTagsAndUris)
data["content"] = data["content"].map(stripTagsAndUris)
import string
print(data.head())

def removePunctuation(x):
    # Lowercasing all words
    x = x.lower()
    # Removing non ASCII chars
    #x = re.sub(r'[^\x00-\x7f]',r' ',x)
    # Removing (replacing with empty spaces actually) all the punctuations
    return re.sub("["+string.punctuation+"]", " ", x)

# point questionmarks etc
data["title"] = data["title"].map(removePunctuation)
data["content"] = data["content"].map(removePunctuation)
#data = clean_dataframe(data)
data.to_csv('cleandata.csv')

In [ ]:
from gensim.parsing import PorterStemmer
global_stemmer = PorterStemmer()
 
class StemmingHelper(object):
    """
    Class to aid the stemming process - from word to stemmed form,
    and vice versa.
    The 'original' form of a stemmed word will be returned as the
    form in which its been used the most number of times in the text.
    """
 
    #This reverse lookup will remember the original forms of the stemmed
    #words
    word_lookup = {}
 
    @classmethod
    def stem(cls, word):
        """
        Stems a word and updates the reverse lookup.
        """
 
        #Stem the word
        stemmed = global_stemmer.stem(word)
 
        #Update the word lookup
        if stemmed not in cls.word_lookup:
            cls.word_lookup[stemmed] = {}
        cls.word_lookup[stemmed][word] = (
            cls.word_lookup[stemmed].get(word, 0) + 1)
 
        return stemmed
 
    @classmethod
    def original_form(cls, word):
        """
        Returns original form of a word given the stemmed version,
        as stored in the word lookup.
        """
 
        if word in cls.word_lookup:
            return max(cls.word_lookup[word].keys(),
                       key=lambda x: cls.word_lookup[word][x])
        else:
            return word
        
#data['title']=data['title'].replace('?',' ?')
data.head(5)# Summary about tags

In [ ]:
from gensim.models import Word2Vec
from six import iteritems
import numpy as np
#import pyemd

wordlist_wv = []
wordlist_li = []
tel=0
for col in ['title', 'tags','content']:
    for sentence in data[col].items():
        if tel/700==round(tel/700):
            print(sentence[1])
        tel+=1
        word_list = str(sentence[1]).split(' ')
        wordlist_wv.append(word_list)
        wordlist_li=np.concatenate((wordlist_li, word_list), axis=0)
wordlist_df=pd.DataFrame(wordlist_li)
wordlist_df
wordmodel = Word2Vec(wordlist_wv , size=100, window=5, min_count=0, workers=5,negative=0,hs=1)
word_vectors = wordmodel.wv
print(wordmodel)

tel=0
for col in ['title']:
    for sentence in data[col].items():
        if tel/250==round(tel/250):
            print(wordmodel.score([str(sentence[1]).split()]),sentence[1])
        tel+=1

		
		
		
		
		
		
		
		
		
		#vector=wmb['sex']
#print(vector)
#wmb.most_similar(positive=['Berlin', 'Japan'], negative=['Germany'])

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
#Z_ matrix out of train_data content

word=pd.DataFrame([i for i in range(len(wordlist_li))], index=wordlist_li)
word.columns=['id']
word.index.name='index'
word.index.unique()

#tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=1,vocabulary=worddf.as_matrix())
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=1)
Z_ = tfidf_vectorizer.fit_transform(data['content'])
Z_names = tfidf_vectorizer.get_feature_names()
Z_voca=pd.DataFrame.from_dict(tfidf_vectorizer.vocabulary_,orient='index') # dict
Z_idfv=tfidf_vectorizer.idf_ # array, shape = [n_features], or None
Z_stop=tfidf_vectorizer.stop_words_ # : set
# Fuse the  Word vector from Word2Vec with the Z_names form TFIDF
Z_voca.index.name='index'

#doe de merge
Vt_Z = [wordmodel[index] for index in Z_names if index in word_vectors.vocab]
Vt_voca = [index for index in Z_names if index in word_vectors.vocab]
#index - nummer verwisselen
Z_voca_ni=pd.DataFrame(Z_voca.index.values)

In [ ]:
xyz=2
print('Monografie___',data.iloc[xyz].content)
print('Tagwords_____',data.iloc[xyz].tags)


#Dtemp=pd.DataFrame(xi.todense())
Dtemp=pd.DataFrame(Z_[xyz].todense())
print(Dtemp[Dtemp>0].count())
Dtemp_Zv=Dtemp.append(Z_voca_ni.T)
DtZ=Dtemp_Zv.T
DtZ.columns=['rf','term']
DtZ.sort('rf',ascending=[False])
DtZ_=pd.DataFrame([])
DtZ_voc=pd.DataFrame([])
DtZ_=pd.DataFrame([])
DtZ_voc=pd.DataFrame(['n','term'])
for xi in DtZ.index:
  if DtZ.iloc[xi].rf>0:
      tempw=DtZ.iloc[xi].term
      if tempw in word_vectors.vocab:
          Tempmat=pd.DataFrame( DtZ.iloc[xi].rf*(wordmodel[tempw]) )
          DtZ_voc= DtZ_voc.append([xi,tempw])
          DtZ_= DtZ_.append(Tempmat.T)
          #DtZ_=np.reshape(DtZ_, (int(round(len(DtZ_)/100)),100))
#print(wordmodel[tempw])
temparr=DtZ_voc.as_matrix()
temparr=np.reshape(temparr, (len(temparr)/2,-1))  
DtZ_voc=pd.DataFrame(temparr)
DtZ_voc.columns=['nr','term']
#Lis of words in tekst
ListOfWords=DtZ.select(lambda s: DtZ.ix[s].rf>0)

from gensim.models import Word2Vec
from six import iteritems
import numpy as np
import matplotlib.pyplot as plt

def plotcorr (DVt,txt,YN):
    Dcorr=DVt.dot(DVt.T)/(abs(DVt).dot(abs(DVt.T)))
    if YN==1:
        plt.figure(figsize=(14,12))
        plt.pcolor(np.array(Dcorr), cmap='plasma')
        plt.colorbar()
        plt.yticks(0.5 + np.arange(len(DtZ_voc)), DtZ_voc)
        plt.xticks(0.5 + np.arange(len(DtZ_voc)), DtZ_voc)
    superwoord=Dcorr[Dcorr>0.85].sum()
    lengte=len(superwoord)
    supdf=pd.DataFrame(superwoord.values,index=range(0,lengte))
    ddt=DtZ_voc.T.append(supdf.T)
    dtt=ddt.T
    
    resultaat=pd.merge(dtt, ListOfWords, how='left', on=['term', 'term'])
    resultaat['multi']=resultaat['rf']*resultaat[0]
    print(resultaat.sort('multi',ascending=0)[:5])
    prnttmp=''
    for wn in  ( resultaat.sort('multi',ascending=0)[:5]['term'].values ):
        prnttmp+=' '+wn
    print(prnttmp)

    if YN==1:
        return plt.show()
    else:
        return prnttmp


plotcorr (DtZ_,DtZ_voc,1)